In [ ]:
import cv2
import numpy as np

# Define HSV color range for each light
def define_color_ranges():
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    lower_yellow = np.array([20, 150, 150])  # Adjusted yellow lower bounds
    upper_yellow = np.array([40, 255, 255])

    lower_green = np.array([40, 50, 50])
    upper_green = np.array([90, 255, 255])

    return (lower_red1, upper_red1), (lower_red2, upper_red2), (lower_yellow, upper_yellow), (lower_green, upper_green)

# Function to detect the color of the traffic light and return contours
def detect_traffic_light(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    (lower_red1, upper_red1), (lower_red2, upper_red2), (lower_yellow, upper_yellow), (lower_green, upper_green) = define_color_ranges()

    red_mask1 = cv2.inRange(hsv_frame, lower_red1, upper_red1)
    red_mask2 = cv2.inRange(hsv_frame, lower_red2, upper_red2)
    red_mask = red_mask1 | red_mask2
    yellow_mask = cv2.inRange(hsv_frame, lower_yellow, upper_yellow)
    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)

    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.erode(red_mask, kernel, iterations=1)
    red_mask = cv2.dilate(red_mask, kernel, iterations=1)

    green_mask = cv2.erode(green_mask, kernel, iterations=1)
    green_mask = cv2.dilate(green_mask, kernel, iterations=1)

    yellow_mask = cv2.erode(yellow_mask, kernel, iterations=1)
    yellow_mask = cv2.dilate(yellow_mask, kernel, iterations=1)

    contours_red, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_yellow, _ = cv2.findContours(yellow_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_green, _ = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    color = "None"
    contours = None

    if len(contours_red) > 0 and max([cv2.contourArea(c) for c in contours_red]) > 500: #reduced area for better detection
        color = "Red, Stop"
        contours = contours_red
    elif len(contours_yellow) > 0 and max([cv2.contourArea(c) for c in contours_yellow]) > 500: #reduced area for better detection
        color = "Yellow, Slow Down"
        contours = contours_yellow
    elif len(contours_green) > 0 and max([cv2.contourArea(c) for c in contours_green]) > 500: #reduced area for better detection
        color = "Green, Go"
        contours = contours_green

    return color, contours

# Function to draw a square around the detected traffic light
def draw_square(frame, color, contours):
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        if color == "Red, Stop":
            box_color = (0, 0, 255)
        elif color == "Yellow, Slow Down":
            box_color = (0, 255, 255)
        elif color == "Green, Go":
            box_color = (0, 255, 0)
        else:
            box_color = (255, 255, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 5)

    return frame

# Main function to read from video file and detect traffic light
def main(video_path):
    cap = cv2.VideoCapture(video_path)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output_traffic_light_detection.mp4', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        traffic_light_color, contours = detect_traffic_light(frame)

        frame = draw_square(frame, traffic_light_color, contours)

        cv2.putText(frame, f'Traffic Light: {traffic_light_color}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print("Video processing complete. Output saved as 'output_traffic_light_detection.mp4'")

if __name__ == "__main__":
    video_path = '/content/21165-315405392_tiny.mp4'  # Replace with your video file path
    main(video_path)

Video processing complete. Output saved as 'output_traffic_light_detection.mp4'


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow # Import cv2_imshow


def detect_lanes(frame):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges using Canny edge detector
    edges = cv2.Canny(blurred, 50, 150)

    # Define a region of interest (ROI)
    height, width = edges.shape
    roi_vertices = [
        (0, height),
        (width // 2, height // 2),  # Adjust this point for your ROI
        (width, height),
    ]
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, [np.array(roi_vertices, np.int32)], 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Apply Hough transform to detect lines
    lines = cv2.HoughLinesP(
        masked_edges,
        rho=2,
        theta=np.pi / 180,
        threshold=100,  # Adjust this threshold
        minLineLength=40,  # Adjust this value
        maxLineGap=20,  # Adjust this value
    )

    # Draw the detected lines on the original frame
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

    # Combine the line image with the original frame
    result = cv2.addWeighted(frame, 0.8, line_image, 1, 0)

    return result

# Load the video
video_path = "/content/80395-572395743_tiny.mp4"  # Replace with your video path
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect lanes in the frame
    processed_frame = detect_lanes(frame)

    # Display the processed frame using cv2_imshow
    cv2_imshow(processed_frame) # Use cv2_imshow instead of cv2.imshow


cap.release()


In [ ]:
import cv2
import numpy as np

# Define HSV color range for traffic lights
def define_color_ranges():
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    lower_yellow = np.array([20, 150, 150])
    upper_yellow = np.array([40, 255, 255])

    lower_green = np.array([40, 50, 50])
    upper_green = np.array([90, 255, 255])

    return (lower_red1, upper_red1), (lower_red2, upper_red2), (lower_yellow, upper_yellow), (lower_green, upper_green)

# Detect traffic lights
def detect_traffic_light(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    (lr1, ur1), (lr2, ur2), (ly, uy), (lg, ug) = define_color_ranges()

    red_mask = cv2.inRange(hsv_frame, lr1, ur1) | cv2.inRange(hsv_frame, lr2, ur2)
    yellow_mask = cv2.inRange(hsv_frame, ly, uy)
    green_mask = cv2.inRange(hsv_frame, lg, ug)

    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.dilate(cv2.erode(red_mask, kernel, iterations=1), kernel, iterations=1)
    yellow_mask = cv2.dilate(cv2.erode(yellow_mask, kernel, iterations=1), kernel, iterations=1)
    green_mask = cv2.dilate(cv2.erode(green_mask, kernel, iterations=1), kernel, iterations=1)

    contours_red, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_yellow, _ = cv2.findContours(yellow_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_green, _ = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    color = "None"
    contours = None

    if contours_red and max([cv2.contourArea(c) for c in contours_red]) > 500:
        color, contours = "Red, Stop", contours_red
    elif contours_yellow and max([cv2.contourArea(c) for c in contours_yellow]) > 500:
        color, contours = "Yellow, Slow Down", contours_yellow
    elif contours_green and max([cv2.contourArea(c) for c in contours_green]) > 500:
        color, contours = "Green, Go", contours_green

    return color, contours

# Draw bounding box for traffic light
def draw_traffic_light_box(frame, color, contours):
    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)
        colors = {
            "Red, Stop": (0, 0, 255),
            "Yellow, Slow Down": (0, 255, 255),
            "Green, Go": (0, 255, 0),
            "None": (255, 255, 255)
        }
        cv2.rectangle(frame, (x, y), (x+w, y+h), colors.get(color, (255, 255, 255)), 5)
    return frame

# Lane detection
def detect_lanes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    height, width = edges.shape
    roi_vertices = [(0, height), (width // 2, height // 2), (width, height)]
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, [np.array(roi_vertices, np.int32)], 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked_edges, 2, np.pi / 180, 100, minLineLength=40, maxLineGap=20)
    line_image = np.zeros_like(frame)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

    result = cv2.addWeighted(frame, 0.8, line_image, 1, 0)
    return result

# Main video processing function
def main(video_path):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output_combined_detection.mp4', fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        traffic_light_color, contours = detect_traffic_light(frame)
        frame = draw_traffic_light_box(frame, traffic_light_color, contours)
        frame = detect_lanes(frame)

        cv2.putText(frame, f'Traffic Light: {traffic_light_color}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    print("Processing complete. Output saved as 'output_combined_detection.mp4'")

# Example usage
if __name__ == "__main__":
    video_path = '/content/sample_video.mp4'  # Replace with your video path
    main(video_path)


Processing complete. Output saved as 'output_combined_detection.mp4'


In [ ]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import tempfile
import os

# Define HSV color ranges
def define_color_ranges():
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    lower_yellow = np.array([20, 150, 150])
    upper_yellow = np.array([40, 255, 255])
    lower_green = np.array([40, 50, 50])
    upper_green = np.array([90, 255, 255])
    return (lower_red1, upper_red1), (lower_red2, upper_red2), (lower_yellow, upper_yellow), (lower_green, upper_green)

# Detect traffic light
def detect_traffic_light(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    (lr1, ur1), (lr2, ur2), (ly, uy), (lg, ug) = define_color_ranges()

    red_mask = cv2.inRange(hsv_frame, lr1, ur1) | cv2.inRange(hsv_frame, lr2, ur2)
    yellow_mask = cv2.inRange(hsv_frame, ly, uy)
    green_mask = cv2.inRange(hsv_frame, lg, ug)

    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
    yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_OPEN, kernel)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)

    contours_red = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    contours_yellow = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    contours_green = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    color = "None"
    contours = None
    min_area = 500

    for cnts, col in [(contours_red, "Red, Stop"),
                     (contours_yellow, "Yellow, Slow Down"),
                     (contours_green, "Green, Go")]:
        if cnts:
            largest = max(cnts, key=cv2.contourArea)
            if cv2.contourArea(largest) > min_area:
                color, contours = col, cnts
                break

    return color, contours

# Draw bounding box
def draw_traffic_light_box(frame, color, contours):
    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)
        colors = {
            "Red, Stop": (0, 0, 255),
            "Yellow, Slow Down": (0, 255, 255),
            "Green, Go": (0, 255, 0)
        }
        cv2.rectangle(frame, (x, y), (x + w, y + h), colors.get(color, (255, 255, 255)), 3)
        cv2.putText(frame, color, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, colors.get(color, (255, 255, 255)), 2)
    return frame

# Detect lanes
def detect_lanes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    height, width = edges.shape
    mask = np.zeros_like(edges)
    roi = np.array([[(0, height), (width//2, height//2), (width, height)]], dtype=np.int32)
    cv2.fillPoly(mask, roi, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked_edges, 2, np.pi/180, 100, minLineLength=40, maxLineGap=20)
    line_image = np.zeros_like(frame)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 255), 5)

    return cv2.addWeighted(frame, 0.8, line_image, 1, 0)

# Streamlit UI
st.title("🚦 Traffic Light & Lane Detection 🛣️")
st.write("Upload a video or image to detect traffic lights and lanes")

uploaded_file = st.file_uploader("Choose a file", type=["mp4", "avi", "mov", "jpg", "png"])

if uploaded_file is not None:
    tfile = tempfile.NamedTemporaryFile(delete=False)
    tfile.write(uploaded_file.read())

    if uploaded_file.type.startswith('image'):
        # Process image
        frame = cv2.imread(tfile.name)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        color, contours = detect_traffic_light(frame)
        frame = draw_traffic_light_box(frame, color, contours)
        frame = detect_lanes(frame)

        st.image(frame, caption="Processed Image", use_column_width=True)
        st.success(f"Detected: {color if color != 'None' else 'No traffic light detected'}")

    else:
        # Process video
        cap = cv2.VideoCapture(tfile.name)
        stframe = st.empty()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            color, contours = detect_traffic_light(frame)
            frame = draw_traffic_light_box(frame, color, contours)
            frame = detect_lanes(frame)

            stframe.image(frame, caption="Processing...", use_column_width=True)

        cap.release()
        st.success("Video processing complete!")

    os.unlink(tfile.name)

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼your url is: https://funny-cases-cover.loca.lt
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:27163 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙

In [9]:
# Install dependencies if not already installed
!pip install -q ipywidgets opencv-python-headless

import cv2
import numpy as np
import io
import os
from base64 import b64encode
from IPython.display import display, HTML
import ipywidgets as widgets
from google.colab import files

# Define HSV color ranges
def define_color_ranges():
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    lower_yellow = np.array([20, 150, 150])
    upper_yellow = np.array([40, 255, 255])
    lower_green = np.array([40, 50, 50])
    upper_green = np.array([90, 255, 255])
    return (lower_red1, upper_red1), (lower_red2, upper_red2), (lower_yellow, upper_yellow), (lower_green, upper_green)

# Traffic light detection
def detect_traffic_light(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    (lr1, ur1), (lr2, ur2), (ly, uy), (lg, ug) = define_color_ranges()
    red_mask = cv2.inRange(hsv, lr1, ur1) | cv2.inRange(hsv, lr2, ur2)
    yellow_mask = cv2.inRange(hsv, ly, uy)
    green_mask = cv2.inRange(hsv, lg, ug)

    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.dilate(cv2.erode(red_mask, kernel), kernel)
    yellow_mask = cv2.dilate(cv2.erode(yellow_mask, kernel), kernel)
    green_mask = cv2.dilate(cv2.erode(green_mask, kernel), kernel)

    contours_red, _ = cv2.findContours(red_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_yellow, _ = cv2.findContours(yellow_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_green, _ = cv2.findContours(green_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    color = "None"
    contours = None

    if contours_red and max([cv2.contourArea(c) for c in contours_red]) > 500:
        return "Red, Stop", contours_red
    elif contours_yellow and max([cv2.contourArea(c) for c in contours_yellow]) > 500:
        return "Yellow, Slow Down", contours_yellow
    elif contours_green and max([cv2.contourArea(c) for c in contours_green]) > 500:
        return "Green, Go", contours_green
    else:
        return color, contours

# Draw bounding box
'''def draw_traffic_light_box(frame, color, contours):
    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)
        color_map = {
            "Red, Stop": (0, 0, 255),
            "Yellow, Slow Down": (0, 255, 255),
            "Green, Go": (0, 255, 0),
            "None": (255, 255, 255)
        }
        cv2.rectangle(frame, (x, y), (x+w, y+h), color_map.get(color, (255,255,255)), 4)
    return frame
'''
def detect_traffic_light(frame):
    # Focus on upper part of the image
    height, width, _ = frame.shape
    roi = frame[0:int(height * 0.5), :]  # Top half only
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    (lr1, ur1), (lr2, ur2), (ly, uy), (lg, ug) = define_color_ranges()

    # Color masks
    red_mask = cv2.inRange(hsv, lr1, ur1) | cv2.inRange(hsv, lr2, ur2)
    yellow_mask = cv2.inRange(hsv, ly, uy)
    green_mask = cv2.inRange(hsv, lg, ug)

    # Morphological operations
    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.dilate(cv2.erode(red_mask, kernel), kernel)
    yellow_mask = cv2.dilate(cv2.erode(yellow_mask, kernel), kernel)
    green_mask = cv2.dilate(cv2.erode(green_mask, kernel), kernel)

    # Circle detection helper
    def detect_circles(mask):
        circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                                   param1=50, param2=15, minRadius=5, maxRadius=50)
        return circles

    # Try to detect circles in each color
    red_circles = detect_circles(red_mask)
    yellow_circles = detect_circles(yellow_mask)
    green_circles = detect_circles(green_mask)

    if red_circles is not None:
        return "Red, Stop", [np.array([[int(c[0]), int(c[1])]]) for c in red_circles[0]]
    elif yellow_circles is not None:
        return "Yellow, Slow Down", [np.array([[int(c[0]), int(c[1])]]) for c in yellow_circles[0]]
    elif green_circles is not None:
        return "Green, Go", [np.array([[int(c[0]), int(c[1])]]) for c in green_circles[0]]
    else:
        return "None", None

# Lane detection
def detect_lanes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)

    height, width = edges.shape
    roi = np.array([[(0, height), (width // 2, height // 2), (width, height)]], dtype=np.int32)
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, roi, 255)
    masked = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked, 2, np.pi/180, 100, minLineLength=40, maxLineGap=20)
    line_img = np.zeros_like(frame)

    if lines is not None:
        for l in lines:
            x1, y1, x2, y2 = l[0]
            cv2.line(line_img, (x1, y1), (x2, y2), (0,0,255), 5)
    return cv2.addWeighted(frame, 0.8, line_img, 1, 0)

# Main video processing function
def process_video(input_path, output_path='output_result.mp4'):
    cap = cv2.VideoCapture(input_path)
    width, height = int(cap.get(3)), int(cap.get(4))
    fps = int(cap.get(5))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        color, contours = detect_traffic_light(frame)
        frame = draw_traffic_light_box(frame, color, contours)
        frame = detect_lanes(frame)
        cv2.putText(frame, f'Traffic Light: {color}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        out.write(frame)

    cap.release()
    out.release()

# Display video HTML
def display_video(file_path):
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""<video width=700 controls><source src="{data_url}" type="video/mp4"></video>"""))

# Upload widget
upload_widget = widgets.FileUpload(accept='.mp4', multiple=False)
process_button = widgets.Button(description="Process Video")
output_area = widgets.Output()

def on_process_clicked(b):
    with output_area:
        output_area.clear_output()
        if upload_widget.value:
            uploaded_filename = list(upload_widget.value.keys())[0]
            uploaded_data = upload_widget.value[uploaded_filename]['content']
            input_path = f'/content/{uploaded_filename}'
            with open(input_path, 'wb') as f:
                f.write(uploaded_data)

            print("Processing...")
            process_video(input_path)
            print("Done! Here's the result:")
            display_video("output_result.mp4")
        else:
            print("Please upload a video first.")

process_button.on_click(on_process_clicked)

# Display frontend UI
display(widgets.VBox([widgets.Label("Upload a .mp4 dashcam video:"), upload_widget, process_button, output_area]))

In [ ]:
# Install dependencies if not already installed
!pip install -q ipywidgets opencv-python-headless

import cv2
import numpy as np
import io
import os
from base64 import b64encode
from IPython.display import display, HTML
import ipywidgets as widgets
from google.colab import files

# Traffic light memory state
class TrafficLightMemory:
    def __init__(self):
        self.last_color = "None"
        self.frame_count = 0
        self.green_hold_duration = 20  # Hold green signal for 20 frames

memory = TrafficLightMemory()

# Define HSV color ranges
def define_color_ranges():
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])
    lower_yellow = np.array([20, 150, 150])
    upper_yellow = np.array([40, 255, 255])
    lower_green = np.array([40, 50, 50])
    upper_green = np.array([90, 255, 255])
    return (lower_red1, upper_red1), (lower_red2, upper_red2), (lower_yellow, upper_yellow), (lower_green, upper_green)

# Traffic light detection using circle detection
def detect_traffic_light(frame):
    height, width, _ = frame.shape
    roi = frame[0:int(height * 0.5), :]  # Top half only
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    (lr1, ur1), (lr2, ur2), (ly, uy), (lg, ug) = define_color_ranges()
    red_mask = cv2.inRange(hsv, lr1, ur1) | cv2.inRange(hsv, lr2, ur2)
    yellow_mask = cv2.inRange(hsv, ly, uy)
    green_mask = cv2.inRange(hsv, lg, ug)

    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.dilate(cv2.erode(red_mask, kernel), kernel)
    yellow_mask = cv2.dilate(cv2.erode(yellow_mask, kernel), kernel)
    green_mask = cv2.dilate(cv2.erode(green_mask, kernel), kernel)

    def detect_circles(mask):
        circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                                   param1=50, param2=15, minRadius=5, maxRadius=50)
        return circles

    red_circles = detect_circles(red_mask)
    yellow_circles = detect_circles(yellow_mask)
    green_circles = detect_circles(green_mask)

    if red_circles is not None:
        return "Red, Stop", [np.array([[int(c[0]), int(c[1])]]) for c in red_circles[0]]
    elif yellow_circles is not None:
        return "Yellow, Slow Down", [np.array([[int(c[0]), int(c[1])]]) for c in yellow_circles[0]]
    elif green_circles is not None:
        return "Green, Go", [np.array([[int(c[0]), int(c[1])]]) for c in green_circles[0]]
    else:
        return "None", None

# Draw bounding box (optional)
def draw_traffic_light_box(frame, color, contours):
    if contours:
        largest = max(contours, key=lambda cnt: cv2.boundingRect(cnt)[2] * cv2.boundingRect(cnt)[3])
        x, y, w, h = cv2.boundingRect(largest)
        color_map = {
            "Red, Stop": (0, 0, 255),
            "Yellow, Slow Down": (0, 255, 255),
            "Green, Go": (0, 255, 0),
            "None": (255, 255, 255)
        }
        cv2.rectangle(frame, (x, y), (x + w, y + h), color_map.get(color, (255, 255, 255)), 4)
    return frame

# Lane detection
def detect_lanes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    height, width = edges.shape
    roi = np.array([[(0, height), (width // 2, height // 2), (width, height)]], dtype=np.int32)
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, roi, 255)
    masked = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked, 2, np.pi / 180, 100, minLineLength=40, maxLineGap=20)
    line_img = np.zeros_like(frame)

    if lines is not None:
        for l in lines:
            x1, y1, x2, y2 = l[0]
            cv2.line(line_img, (x1, y1), (x2, y2), (0, 0, 255), 5)
    return cv2.addWeighted(frame, 0.8, line_img, 1, 0)

# Main video processing function
def process_video(input_path, output_path='output_result.mp4'):
    cap = cv2.VideoCapture(input_path)
    width, height = int(cap.get(3)), int(cap.get(4))
    fps = int(cap.get(5))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        detected_color, contours = detect_traffic_light(frame)

        # Memory logic to hold green signal longer
        if detected_color == "Green, Go":
            memory.last_color = "Green, Go"
            memory.frame_count = memory.green_hold_duration
        elif detected_color != "None":
            memory.last_color = detected_color
            memory.frame_count = 0
        elif memory.frame_count > 0:
            memory.frame_count -= 1
            detected_color = memory.last_color
        else:
            memory.last_color = "None"
            detected_color = "None"

        # Draw detection and lane lines
        frame = draw_traffic_light_box(frame, detected_color, contours)
        frame = detect_lanes(frame)

        # Display result text
        cv2.putText(frame, f'Traffic Light: {detected_color}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        out.write(frame)

    cap.release()
    out.release()

# Display video in notebook
def display_video(file_path):
    mp4 = open(file_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""<video width=700 controls><source src="{data_url}" type="video/mp4"></video>"""))

# Upload and process UI
upload_widget = widgets.FileUpload(accept='.mp4', multiple=False)
process_button = widgets.Button(description="Process Video")
output_area = widgets.Output()

def on_process_clicked(b):
    with output_area:
        output_area.clear_output()
        if upload_widget.value:
            uploaded_filename = list(upload_widget.value.keys())[0]
            uploaded_data = upload_widget.value[uploaded_filename]['content']
            input_path = f'/content/{uploaded_filename}'
            with open(input_path, 'wb') as f:
                f.write(uploaded_data)

            print("Processing...")
            process_video(input_path)
            print("Done! Here's the result:")
            display_video("output_result.mp4")
        else:
            print("Please upload a video first.")

process_button.on_click(on_process_clicked)

# Display UI
display(widgets.VBox([widgets.Label("Upload a .mp4 dashcam video:"), upload_widget, process_button, output_area]))

files.download('output_result.mp4')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.8 MB/s eta 0:00:00


FileNotFoundError: Cannot find file: output_result.mp4